<a href="https://colab.research.google.com/github/BorowskiKacper/llm_multiagent_debate/blob/main/Multi_Agent_Review_Board.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# README

# Multi agent review board notebook using "mmlu" data (Source code has "geography", "gsm", "math", and "mmlu" data)
# Source code: https://github.com/composable-models/llm_multiagent_debate

# It's highly suggested that you CONFIGURE certain parameters, especially pipelines and agent configs.
# Use the third code block/cell to do so.

In [1]:
# Google Colab setup
!pip install transformers accelerate -q


In [ ]:
# CONFIGURE rounds, questions, pipelines, and agent configs in this cell
from transformers import pipeline
import torch

# =====ROUNDS & QUESTIONS=====
rounds = 2
questions = 5

# =====GENERATION PIPELINES=====
# NOTE: You can create pipelines from multiple models, not just one.
# But beware that you have a limited amount of RAM and VRAM to work withRuntime -> View Resources
model_id="meta-llama/Llama-3.2-1B-Instruct"
pipe = pipeline(
    "text-generation",
    model=model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto"
)

# =====AGENTS=====
# Currently each agent uses the same model/pipeline, but different temperatures
agent_configs = [{"pipe": pipe, "temp": 0.1}, {"pipe": pipe, "temp": 0.7}, {"pipe": pipe, "temp": 1.5}]

In [2]:
from glob import glob
import pandas as pd
import json
import time
import random


In [3]:
# Load MMLU data and store data frames
!curl "https://people.eecs.berkeley.edu/~hendrycks/data.tar" | tar -xvf -

tasks = glob("./data/test/*.csv")
dfs = [pd.read_csv(task) for task in tasks]

len(dfs)


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0data/
data/README.txt
data/val/
data/val/high_school_geography_val.csv
data/val/astronomy_val.csv
data/val/conceptual_physics_val.csv
data/val/medical_genetics_val.csv
data/val/high_school_computer_science_val.csv
data/val/business_ethics_val.csv
data/val/anatomy_val.csv
data/val/global_facts_val.csv
data/val/philosophy_val.csv
data/val/high_school_government_and_politics_val.csv
data/val/college_physics_val.csv
data/val/miscellaneous_val.csv
data/val/machine_learning_val.csv
data/val/high_school_mathematics_val.csv
data/val/high_school_biology_val.csv
data/val/abstract_algebra_val.csv
data/val/college_medicine_val.csv
data/val/clinical_knowledge_val.csv
data/val/electrical_engineering_val.csv
data/val/econometrics_val.csv
data/val/world_religions_val.cs

57

In [12]:
def construct_message(agents, question, idx):
    if len(agents) == 0:
        return {"role": "user", "content": "Can you double check that your answer is correct. Put your final answer in the form (X) at the end of your response."}

    prefix_string = "These are the solutions to the problem from other agents: "

    for agent in agents:
        agent_response = agent[idx]["content"]
        response = "\n\n One agent solution: ```{}```".format(agent_response)

        prefix_string = prefix_string + response

    prefix_string = prefix_string + """\n\n Using the reasoning from other agents as additional advice, can you give an updated answer? Examine your solution and that other agents step by step. Put your answer in the form (X) at the end of your response.""".format(question)
    return {"role": "user", "content": prefix_string}


def construct_assistant_message(completion):
    # content = completion["choices"][0]["message"]["content"]
    content = completion[0]["generated_text"][-1]["content"]
    return {"role": "assistant", "content": content}


def generate_answer(agent_config, agent_context):
    # try:

    #     completion = openai.ChatCompletion.create(
    #               model="gpt-3.5-turbo-0301",
    #               messages=answer_context,
    #               n=1)
    # except:
    #     print("retrying due to an error......")
    #     time.sleep(20)
    #     return generate_answer(answer_context)
    completion = agent_config["pipe"](
      agent_context,
      max_new_tokens=1024,
      do_sample=True,
      temperature=agent_config["temp"],
      top_p=0.9,
    )


    return completion


def parse_question_answer(df, ix):
    question = df.iloc[ix, 0]
    a = df.iloc[ix, 1]
    b = df.iloc[ix, 2]
    c = df.iloc[ix, 3]
    d = df.iloc[ix, 4]

    question = "Can you answer the following question as accurately as possible? {}: A) {}, B) {}, C) {}, D) {} Explain your answer, putting the answer in the form (X) at the end of your response.".format(question, a, b, c, d)

    answer = df.iloc[ix, 5]

    return question, answer



`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

Device set to use cuda:0


In [15]:
random.seed(0)
response_dict = {}

for i in range(questions):
    df = random.choice(dfs)
    ix = len(df)
    idx = random.randint(0, ix-1)

    question, answer = parse_question_answer(df, idx)

    # agent_contexts = [[{"role": "user", "content": question}] for agent in range(agents)]
    agent_contexts = [[{"role": "user", "content": question}] for agent in range(len(agent_configs))]
    print(f"Question {i+1}/{questions}: {question}")
    print(f"Answer: {answer}")

    for round in range(rounds):
        for i, agent_context in enumerate(agent_contexts):
            print(f"Round: {round + 1}/{rounds} | Agent: {i+1}/{len(agent_configs)}")
            if round != 0:
                agent_contexts_other = agent_contexts[:i] + agent_contexts[i+1:]
                message = construct_message(agent_contexts_other, question, 2 * round - 1)
                agent_context.append(message)

            completion = generate_answer(agent_configs[i], agent_context)

            assistant_message = construct_assistant_message(completion)
            agent_context.append(assistant_message)
            print(assistant_message)

    response_dict[question] = (agent_contexts, answer)

json.dump(response_dict, open("mmlu_{}_{}.json".format(len(agent_configs), rounds), "w"))


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Question 1/5: Can you answer the following question as accurately as possible? In defining the term 'historical materialism', which of the following statements best defines the term 'materialism'?: A) Materialism refers to HM's focus on class and the productive basis of societies, as well as the ways in which social structures have the capacity to change and produce change., B) Materialism refers to the roots of HM in the postmodern schools of thought., C) The term materialism refers to the focus of HM on consumerism and the centrality of economic power in the approach., D) In their rejection of class orientated political interaction, HM scholars refer to wealth instead as the accumulation or access to physical resources, coining the term materialism to describe this approach. Explain your answer, putting the answer in the form (X) at the end of your response.
Answer: A
Round: 1/2 | Agent: 1/3


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


{'role': 'assistant', 'content': "To answer this question accurately, let's break down the key concepts and definitions related to historical materialism.\n\nHistorical materialism is a theoretical framework that emphasizes the role of material conditions in shaping social and economic structures. It is a broad concept that encompasses various aspects, including class, power, and the relationship between the state and society.\n\nA) Materialism refers to HM's focus on class and the productive basis of societies, as well as the ways in which social structures have the capacity to change and produce change. This statement accurately captures the core idea of historical materialism, which emphasizes the importance of material conditions in shaping social and economic relationships.\n\nB) Materialism refers to the roots of HM in the postmodern schools of thought. While historical materialism has its roots in Marxist thought, it is not necessarily tied to postmodern schools of thought. This

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

{'role': 'assistant', 'content': "A study conducted by the University of Zurich (2014) analyzed the opinions of 1,050 individuals in Turkey regarding governance. As a result of their study, the majority agreed that the state was 'run for the benefit of all people'.\n\nAbout 66.2% of those who participated in the 2014 study stated that the state is 'run for the benefit of all people'. Hence, (C) is the correct answer.\n\n(66.2% out of 1,050 is: 66.2 / 1,050, which approximates 5.03%)"}
Round: 2/2 | Agent: 1/3


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


{'role': 'assistant', 'content': "To find the correct answer, let's analyze the information from the different sources:\n\n1. A survey conducted by the Turkish National Intelligence Agency (TİKA) found that 52% of Turkish citizens believed that the state is run for the benefit of all the people.\n\n2. A study conducted by the University of Zurich (2014) analyzed the opinions of 1,050 individuals in Turkey regarding governance. As a result of their study, the majority agreed that the state was 'run for the benefit of all people'.\n\n3. About 66.2% of those who participated in the 2014 study stated that the state is 'run for the benefit of all people'.\n\nNow, let's calculate the percentage of people from Turkey who agree that the state is run for the benefit of all the people:\n\n- The Turkish National Intelligence Agency (TİKA) survey: 52% (out of 100%)\n- The University of Zurich study: 66.2% (out of 1,050)\n- The 2014 study: 66.2% (out of 1,050)\n\nTo find the overall percentage, we 

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


{'role': 'assistant', 'content': "Here's the step-by-step analysis:\n\n1. The Turkish Statistical Institute (TÜIK) survey (2019) found that 66% of the Turkish population agreed that the state is run for the benefit of all the people.\n\n2. A study conducted by the University of Zurich (2014) analyzed the opinions of 1,050 individuals in Turkey regarding governance. The majority agreed that the state was 'run for the benefit of all people'.\n\n3. The study found that 66.2% of those who participated in the 2014 study stated that the state is 'run for the benefit of all people'.\n\n4. To find the percentage of people from Turkey who agree that the state is run for the benefit of all the people, we need to consider the survey conducted by the Turkish Statistical Institute (TÜIK) as well.\n\n5. Since the study by the University of Zurich (2014) is based on a sample of 1,050 individuals, we can assume that the survey conducted by the Turkish Statistical Institute (TÜIK) is representative of 